In [36]:
import pandas as pd
import numpy as np

In [37]:
train=pd.read_csv('train.csv')
train.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [38]:
test=pd.read_csv('test.csv')
test.head()

,User_ID,Description,Browser_Used,Device_Used
0,id80132,Looking for a motel in close proximity to TV t...,Firefox,Mobile
1,id80133,Walking distance to Madison Square Garden and ...,InternetExplorer,Desktop
2,id80134,Visited Seattle on business. Spent - nights in...,IE,Tablet
3,id80135,This hotel location is excellent and the rooms...,Edge,Mobile
4,id80136,This hotel is awesome I love the service Antho...,Mozilla,Mobile


In [39]:
train=train.drop(['Browser_Used','Device_Used','User_ID'],axis=1)
test=test.drop(['Browser_Used','Device_Used'],axis=1)
test.head()

,User_ID,Description
0,id80132,Looking for a motel in close proximity to TV t...
1,id80133,Walking distance to Madison Square Garden and ...
2,id80134,Visited Seattle on business. Spent - nights in...
3,id80135,This hotel location is excellent and the rooms...
4,id80136,This hotel is awesome I love the service Antho...


In [40]:
noise_list = ["is", "a", "this", "...","the", "and", "to", "of", "in", "i","that", "it", "on", "you",
               "for", "but", "with", "are", "have", "be", "at", "or", "as", "was", "so", "if", 
              "out"] 
def _remove_noise(input_text):
    words = input_text.split() 
    noise_free_words = [word for word in words if word not in noise_list] 
    noise_free_text = " ".join(noise_free_words) 
    return noise_free_text


In [41]:
test['Description']=test['Description'].apply(lambda x: _remove_noise(x.lower()))
train['Description']=train['Description'].apply(lambda x: _remove_noise(x.lower()))
train.head()

,Description,Is_Response
0,room kind clean had very strong smell dogs. ge...,not happy
1,"stayed crown plaza april -- - april --, ----. ...",not happy
2,booked hotel through hotwire lowest price coul...,not happy
3,stayed here husband sons way an alaska cruise....,happy
4,my girlfriends stayed here celebrate our --th ...,not happy


In [42]:
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()

from nltk.stem.porter import PorterStemmer 
stem = PorterStemmer()

def lexicon_normalization(input_text):
    input_text=lem.lemmatize(input_text, "v")
    input_text=stem.stem(input_text)
    return input_text

def texture(text):
    text=text.split()
    words=[lexicon_normalization(word) for word in text]
    text = " ".join(words) 
    return text  

In [44]:
test['Description']=test['Description'].apply(lambda x: texture(x))
train['Description']=train['Description'].apply(lambda x: texture(x))
train.head()

,Description,Is_Response
0,room kind clean have veri strong smell dogs. g...,not happy
1,"stay crown plaza april -- - april --, ----. st...",not happy
2,book hotel through hotwir lowest price could f...,not happy
3,stay here husband son way an alaska cruise. we...,happy
4,my girlfriend stay here celebr our --th birthd...,not happy


In [45]:
lookup_dict = {'rt':'Retweet', 'dm':'direct message', "awsm" : "awesome", "luv" :"love"}
def _lookup_words(input_text):
    words = input_text.split() 
    new_words = [] 
    for word in words:
        if word.lower() in lookup_dict:
            word = lookup_dict[word.lower()]
        new_words.append(word) 
    new_text = " ".join(new_words) 
    return new_text

In [46]:
test['Description']=test['Description'].apply(lambda x: _lookup_words(x))
train['Description']=train['Description'].apply(lambda x:_lookup_words(x))
train.head()

,Description,Is_Response
0,room kind clean have veri strong smell dogs. g...,not happy
1,"stay crown plaza april -- - april --, ----. st...",not happy
2,book hotel through hotwir lowest price could f...,not happy
3,stay here husband son way an alaska cruise. we...,happy
4,my girlfriend stay here celebr our --th birthd...,not happy


In [59]:
import gensim
from gensim import corpora
doc_clean=[words.split() for words in np.array(train['Description'])]
dictionary = corpora.Dictionary(doc_clean)


In [60]:
term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [ ]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(term_matrix, num_topics=3, id2word = dictionary, passes=50)


In [ ]:
print(ldamodel.print_topics())